In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Sentiment Analysis with RNN

Implementing an RNN that performs Sentiment Analysis I am using a dataset of movie reviews, and a dataset of sentiment labels: positive or negative.

In [1]:
import numpy as np 

with open('/kaggle/input/reviewslabels/reviews.txt','r') as f:
    reviews = f.read()
with open('/kaggle/input/reviewslabels/labels.txt','r') as f:
    labels = f.read()

In [2]:
print(reviews[:2000])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [3]:
print(labels[:10])

positive
n


In [4]:
#Preprocessing
#remove punctuations

from string import punctuation

reviews = reviews.lower()

#remove punctuations
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')

all_text = ' '.join(reviews_split)

# print(all_text[:2000])
# print(type(reviews_split))

words = all_text.split()

print(words[:10])




['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']


In [5]:
from collections import Counter

#build a dictionary of words and counts
counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

vocab_to_int = {c:i for i,c in enumerate(vocab,1)}



#store the reviews as a list

reviews_int = []

for review  in reviews_split:
    reviews_int.append([vocab_to_int[c] for c in review.split()])


In [6]:
# testing the result

#unique words
print(len(vocab_to_int))

#print tokenized review

print(reviews_int[:2])

74072
[[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23], [63, 4, 3, 125, 36, 47, 7472, 1395, 16, 3, 4181, 505, 45, 17, 3, 622, 134, 12, 6, 3, 1279, 457, 4, 1721, 207, 3, 10624, 7373, 300, 6, 667, 83, 35, 2116, 1086, 2989, 34, 1, 898, 46417, 4, 8, 13, 5096, 464, 8, 2656, 1721, 1, 221, 57, 17, 58, 794, 1297, 832, 228, 8, 43, 98, 123, 1469, 59, 147, 38, 1, 963, 142, 29, 667, 123, 1, 13584, 410, 61, 9

In [7]:
#Encode labels to 0 and 1

labels_split = labels.split('\n')
encoded_labels = np.array([1 if x == 'positive' else 0 for x in labels_split])

In [8]:
#preprocessing

review_len = Counter([len(x) for x in reviews_int])

print("Zero-length reviews: {}".format(review_len[0]))
print("Maximum review length: {}".format(max(review_len)))
print("Avg review length: {}".format(sum(review_len)/len(review_len)))
print("Median review length: {}".format(sorted(review_len)[int(len(review_len)/2)]))

Zero-length reviews: 1
Maximum review length: 2514
Avg review length: 539.1170731707317
Median review length: 524


In [9]:
# removing the zero length reviews

print('num of reviews before:', len(reviews_int))

non_zero_idx = [i for i,x in enumerate(reviews_int) if len(reviews_int[i]) != 0 ]

reviews_int = [reviews_int[i] for i in  non_zero_idx]

encoded_labels = np.array([encoded_labels[i] for i in non_zero_idx])

print ('num of reviews after:', len(reviews_int))



num of reviews before: 25001
num of reviews after: 25000


In [10]:
#function to make the reviews same length
# shorter reviews are left padded with zeros
# this will make the reviews_int a 2d array of rows = num of reviews and cols = seq_length

def pad_features(reviews_int, seq_len):
    
    features = np.zeros((len(reviews_int), seq_len),dtype=int)
    
    for i,x in enumerate(reviews_int):
        features[i,-len(x):] = np.array(x)[:seq_len]
    
    return features
        
        
        

In [11]:
seq_length = 200

features = pad_features(reviews_int, seq_len=seq_length)

assert len(features) == len (reviews_int), " The number of rows is different"
assert features.shape[1] == seq_length, " The number of columns is different"

In [12]:
print(features [:20, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [13]:
# spliting the data to training test and validation sets
split_frac = 0.8

split_idx = int(len(features)*split_frac)

train_x, remaining_x = features[:split_idx],features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]


split_idx = int(len(remaining_x)*0.5)

val_x, test_x = remaining_x[:split_idx], remaining_x[split_idx:]
val_y, test_y = remaining_y[:split_idx], remaining_y[split_idx:]


print("training set: {} label {} ".format (train_x.shape, train_y.shape))
print("test set: {} label {} ".format (test_x.shape, test_y.shape))
print("val set: {} label {} ".format (val_x.shape, val_y.shape))



training set: (20000, 200) label (20000,) 
test set: (2500, 200) label (2500,) 
val set: (2500, 200) label (2500,) 


In [15]:
# loading  the data to Dataloader and batching

import torch
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
test_data  = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
val_data = TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))


batch_size = 50

train_loader = DataLoader(train_data, shuffle = True, batch_size= batch_size)
test_loader = DataLoader (test_data, shuffle = True, batch_size = batch_size)
val_loader = DataLoader (val_data, shuffle=True, batch_size = batch_size)


In [16]:
#testing 

jangiter = iter(train_loader)

jangx, jangy = next(jangiter)

print(jangx.size())
print (jangy.size())

print(jangx)
print (jangy)

torch.Size([50, 200])
torch.Size([50])
tensor([[    1,   341,     4,  ...,    75,     5,   122],
        [   11,    20,     6,  ...,   470, 14115,   427],
        [ 1815,    13,   340,  ...,    14,   169,     5],
        ...,
        [    0,     0,     0,  ...,   210,    42,   159],
        [    1,  2797,  2912,  ...,    22,  1411,    13],
        [ 1025,    60,   573,  ...,   109,  3086,    19]])
tensor([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1])


In [17]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("GPU is available")
else:
    print("GPU is not available.Using CPU")
    
    

GPU is available


In [22]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    
    def __init__(self,vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob = 0.5):
        super().__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,n_layers,dropout=drop_prob, batch_first = True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        
        self.sig = nn.Sigmoid()
        
        
    def forward (self,x, hidden):
        
        batch_size = x.size(0)
        
        x = x.long()
        embeds = self.embedding(x)
        
        lstm_out, hidden = self.lstm(embeds,hidden)
        
        lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        sig_out = self.sig(out)
        
        #reshape to batch_size first
        sig_out = sig_out.view (batch_size, -1)
      
        sig_out = sig_out[:, -1] # get last batch of labels, last label will say if it is +ve or -ve
        
        return sig_out, hidden
    
    def init_hidden(self,batch_size):
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [19]:
#define hyper parameters and intantiate the model 

vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [20]:
# defining training hyper parameters

lr = 0.001

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [24]:
epochs = 4 

counter = 0
print_every = 100
clip = 5 

if train_on_gpu:
    net=net.cuda()

net.train()

for e in range(epochs):
    h = net.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter +=1 
        
        if train_on_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
                        
        h = tuple([each.data for each in h])
        
        net.zero_grad()
        
        output, h = net(inputs, h)
        
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()    
        
        #validation 
        if counter % print_every == 0:
            
            
            val_h = net.init_hidden(batch_size)
            net.eval()
            val_losses = []
            
            for inputs, labels in val_loader:
                
                val_h = tuple([each.data for each in val_h])
                
                if train_on_gpu:
                    inputs, labels = inputs.cuda(), labels.cuda()
                    
                output, val_h = net(inputs,val_h)
                
                val_loss = criterion(output.squeeze(), labels.float())
                
                val_losses.append(val_loss.item())
                
            net.train()
            
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))       
            

Epoch: 1/4... Step: 100... Loss: 0.489845... Val Loss: 0.586288
Epoch: 1/4... Step: 200... Loss: 0.563123... Val Loss: 0.579647
Epoch: 1/4... Step: 300... Loss: 0.432164... Val Loss: 0.499122
Epoch: 1/4... Step: 400... Loss: 0.599542... Val Loss: 0.497215
Epoch: 2/4... Step: 500... Loss: 0.401204... Val Loss: 0.496560
Epoch: 2/4... Step: 600... Loss: 0.345174... Val Loss: 0.458867
Epoch: 2/4... Step: 700... Loss: 0.451744... Val Loss: 0.525779
Epoch: 2/4... Step: 800... Loss: 0.302977... Val Loss: 0.451224
Epoch: 3/4... Step: 900... Loss: 0.248862... Val Loss: 0.485913
Epoch: 3/4... Step: 1000... Loss: 0.176518... Val Loss: 0.467932
Epoch: 3/4... Step: 1100... Loss: 0.364825... Val Loss: 0.469616
Epoch: 3/4... Step: 1200... Loss: 0.238338... Val Loss: 0.492017
Epoch: 4/4... Step: 1300... Loss: 0.153911... Val Loss: 0.553701
Epoch: 4/4... Step: 1400... Loss: 0.150469... Val Loss: 0.511068
Epoch: 4/4... Step: 1500... Loss: 0.120247... Val Loss: 0.512971
Epoch: 4/4... Step: 1600... Loss: 

# Testing

In [28]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
 
for inputs, labels in test_loader:
     
    h = tuple([each.data for each in h])

    if train_on_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()
    
    output, h = net(inputs, h)
    
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))     

Test loss: 0.524
Test accuracy: 0.800


# Inference on a test review¶

Giving a user given random review and testing the model for accuracy

In [55]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'
# test_review_neg = 'I wasted my time '


In [30]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 247, 18, 10, 28, 108, 113, 14, 388, 2, 10, 181, 60, 273, 144, 11, 18, 68, 76, 113, 2, 1, 410, 14, 539]]


In [31]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   1 247  18  10  28
  108 113  14 388   2  10 181  60 273 144  11  18  68  76 113   2   1 410
   14 539]]


In [32]:
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 200])


In [34]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")
        

In [35]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.001802
Negative review detected.


In [53]:
# positive test review
# test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

# test_review_pos = 'this is an awesome movie'
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [54]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.987574
Positive review detected!
